In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.models import vgg16, densenet121
from timm import create_model
import time
from tqdm import tqdm
import os
from thop import profile






In [6]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Split the training data into training and validation sets
train_length = int(0.9 * len(train_dataset))
val_length = len(train_dataset) - train_length
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_length, val_length])

trainloader = torch.utils.data.DataLoader(train_subset, batch_size=256, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_subset, batch_size=256, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

models = {
    "VGGNet16": vgg16(num_classes=100).to(device),
    "DenseNet": densenet121(num_classes=100).to(device),
    "EfficientNet": create_model('efficientnet_b0', pretrained=False, num_classes=100).to(device),
    "NFNet": create_model('nfnet_f0', pretrained=False, num_classes=100).to(device), 
    "RegNet": create_model('regnety_002', pretrained=False, num_classes=100).to(device),
    # "InceptionV4": create_model('inception_v4', pretrained=False, num_classes=100).to(device)
    "ResNet-50": create_model('resnet50', pretrained=False, num_classes=100).to(device)

}

criterion = nn.CrossEntropyLoss()
optimizers = {name: optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4) for name, model in models.items()}


In [ ]:
from thop import profile

def compute_flops(models):
    flops_results = {}

    for name, model in models.items():
        print(f"Computing FLOPs for {name}...")

        # Move model to device
        model = model.to(device)

        # Create a random input sample to feed into the model
        input_sample = torch.randn(1, 3, 224, 224).to(device)  # Assuming input size is 224x224x3

        # Compute FLOPs
        macs, params = profile(model, inputs=(input_sample, ))
        flops = macs * 2  # Convert Multiply-Accumulate operations to FLOPs

        flops_results[name] = flops

        print(f"{name} - FLOPs: {flops:.2f}")

    return flops_results

# Call the function
flops_data = compute_flops(models)
print(flops_data)


In [7]:

num_epochs = 1
results = {}
model_save_path = "best_model_weights"  # Directory to save the best model weights

if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

for name, model in models.items():
    print(f"Training {name}...")
    optimizer = optimizers[name]
    best_accuracy = 0.0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    epoch_times = []
    
    # Model Size and Complexity
    total_params = sum(p.numel() for p in model.parameters())
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        start_time = time.time()
        
        # Training phase
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for i, (inputs, labels) in enumerate(tqdm(trainloader, desc="Training", leave=False)):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted_train = outputs.max(1)
            total_train += labels.size(0)
            correct_train += predicted_train.eq(labels).sum().item()
        
        avg_train_loss = running_loss / len(trainloader)
        train_losses.append(avg_train_loss)
        train_accuracy = 100 * correct_train / total_train
        train_accuracies.append(train_accuracy)
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        running_val_loss = 0.0
        
        with torch.no_grad():
            for data in tqdm(valloader, desc="Validation", leave=False):
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                val_loss = criterion(outputs, labels)
                running_val_loss += val_loss.item()
                
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        avg_val_loss = running_val_loss / len(valloader)
        val_losses.append(avg_val_loss)
        
        accuracy = 100 * correct / total
        val_accuracies.append(accuracy)
        
        # Save the model weights if this epoch gives the highest validation accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), os.path.join(model_save_path, f"{name}_best_weights.pth"))
        
        end_time = time.time()
        epoch_duration = end_time - start_time
        epoch_times.append(epoch_duration)
        
        print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%, Time: {epoch_duration:.2f}s")
    
    train_loss_stability = np.std(train_losses)
    
    results[name] = {
        "Best Accuracy": best_accuracy,
        "Training Losses": train_losses,
        "Training Accuracies": train_accuracies,
        "Validation Losses": val_losses,
        "Validation Accuracies": val_accuracies,
        "Epoch Times": epoch_times,
        "Model Parameters": total_params,
        "Training Loss Stability": train_loss_stability
    }


csv_filename = "results.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header
    header = ['Model Name', 'Best Accuracy', 'Model Parameters', 'Training Loss Stability']
    for i in range(num_epochs):
        header.extend([f'Training Loss (Epoch {i+1})', f'Training Accuracy (Epoch {i+1})', f'Validation Loss (Epoch {i+1})', f'Validation Accuracy (Epoch {i+1})', f'Epoch Time (Epoch {i+1})'])
    writer.writerow(header)
    
    # Write model results
    for name, metrics in results.items():
        row = [name, metrics['Best Accuracy'], metrics['Model Parameters'], metrics['Training Loss Stability']]
        for i in range(num_epochs):
            row.extend([metrics['Training Losses'][i], metrics['Training Accuracies'][i], metrics['Validation Losses'][i], metrics['Validation Accuracies'][i], metrics['Epoch Times'][i]])
        writer.writerow(row)

print(f"Results saved to {csv_filename}")

Training ResNet-50...


Epochs:   1%|          | 1/100 [00:20<33:48, 20.49s/it]

Epoch 1/100 - Training Loss: 4.4677, Training Accuracy: 2.92%, Validation Loss: 4.4263, Validation Accuracy: 3.12%, Time: 20.49s


Epochs:   2%|▏         | 2/100 [00:38<30:39, 18.77s/it]

Epoch 2/100 - Training Loss: 4.3397, Training Accuracy: 4.18%, Validation Loss: 4.2968, Validation Accuracy: 4.90%, Time: 17.56s


...

Epochs:  99%|█████████▉| 99/100 [25:13<00:14, 14.37s/it]

Epoch 99/100 - Training Loss: 0.1373, Training Accuracy: 96.19%, Validation Loss: 2.7201, Validation Accuracy: 46.18%, Time: 14.44s


Epochs: 100%|██████████| 100/100 [25:29<00:00, 15.29s/it]


Epoch 100/100 - Training Loss: 0.1360, Training Accuracy: 96.27%, Validation Loss: 2.7126, Validation Accuracy: 46.60%, Time: 15.31s


NameError: name 'csv' is not defined

In [4]:
num_epochs = 100
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    optimizer = optimizers[name]
    best_accuracy = 0.0
    train_losses = []
    val_accuracies = []
    epoch_times = []
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):  # Progress bar for epochs
        start_time = time.time()
        
        # Training phase
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(trainloader, desc="Training", leave=False)):  # Progress bar for training
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        avg_train_loss = running_loss / len(trainloader)
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in tqdm(valloader, desc="Validation", leave=False):  # Progress bar for validation
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        accuracy = 100 * correct / total
        val_accuracies.append(accuracy)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
        
        end_time = time.time()
        epoch_duration = end_time - start_time
        epoch_times.append(epoch_duration)
        
        print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {accuracy:.2f}%, Time: {epoch_duration:.2f}s")
    
    results[name] = {
        "Best Accuracy": best_accuracy,
        "Training Losses": train_losses,
        "Validation Accuracies": val_accuracies,
        "Epoch Times": epoch_times
    }


Training InceptionV4...


Epochs:   0%|          | 0/100 [00:03<?, ?it/s]


RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [8]:
print("Model Performance on CIFAR-100:")
for name, accuracy in results.items():
    print(f"{name}: {accuracy}")


Model Performance on CIFAR-100:
ResNet-50: {'Best Accuracy': 47.52, 'Training Losses': [4.467696498740803, 4.339742814952677, 4.195556648752906, 4.009310928258029, 3.779786069284786, 3.5790853012691843, 3.424670006741177, 3.2937571785666724, 3.1585595567118037, 3.0271827348253946, 2.9139803607355463, 2.7988679910247978, 2.7035583677616986, 2.6087137433615597, 2.5235584527254105, 2.4357141202146355, 2.353095969015902, 2.284189697016369, 2.224384416910735, 2.1486743119629947, 2.093364800919186, 2.0347078747370024, 1.97758130796931, 1.9226011694832281, 1.8701912584629925, 1.8169844448566437, 1.766413583674214, 1.711820525201884, 1.6685159904035656, 1.6233983317559415, 1.5781515389680862, 1.5350030653856017, 1.4904180832884528, 1.4409717402674935, 1.4002107625657862, 1.3513608263297514, 1.3105155602097511, 1.2728829363530332, 1.23476933891123, 1.1892966651781038, 1.1402391344308853, 1.106902283362367, 1.0764154947616837, 1.029984217475761, 0.9914979416538369, 0.9523583833466877, 0.91182904